## Homework 1

In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

### Q1 Downloading data

In [2]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


In [3]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [4]:
df_jan.shape[1]

19

### Q2 Computing duration

In [5]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan["duration"] = df_jan["duration"].dt.total_seconds() / 60

In [6]:
std_duration = df_jan["duration"].std()

In [7]:
print(f"Standard deviation of duration in January: {std_duration:.2f} minutes")

Standard deviation of duration in January: 42.59 minutes


### Q3 Dropping outluiers

In [8]:
df_jan_crop = df_jan[
    (df_jan["duration"] >= 1) &
    (df_jan["duration"] <= 60)
]

In [9]:
print("Fraction of trips with duration between 1 and 60 minutes in January:")
print(f"{len(df_jan_crop) / len(df_jan):.2%}")

Fraction of trips with duration between 1 and 60 minutes in January:
98.12%


### Q4 One-hot encoding 

In [10]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ['trip_distance']
df_jan_crop[categorical] = df_jan_crop[categorical].astype("str")

/tmp/ipykernel_10318/1466682212.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_crop[categorical] = df_jan_crop[categorical].astype("str")


In [11]:
dict_jan = df_jan_crop[categorical ].to_dict(orient="records")

In [12]:
df_jan_crop.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [13]:
transform = DictVectorizer()
X_jan = transform.fit_transform(dict_jan)

In [14]:
print("Feature matrix shape:", X_jan.shape)

Feature matrix shape: (3009173, 515)


### Q5 Training model

In [15]:
y_jan = df_jan_crop["duration"].values
model = LinearRegression()
model.fit(X_jan, df_jan_crop["duration"])
y_pred = model.predict(X_jan)
rmse = mean_squared_error(y_jan, y_pred, squared=False)
print(f"RMSE of the model: {rmse:.2f} minutes")

RMSE of the model: 7.65 minutes


/home/ubuntu/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Q6 Validating the model

In [20]:
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration"] = df_feb["duration"].dt.total_seconds() / 60
df_feb[categorical] = df_feb[categorical].astype("str")



In [21]:
df_feb_crop = df_feb[
    (df_feb["duration"] >= 1) &
    (df_feb["duration"] <= 60)
]

In [22]:
dict_feb = df_feb_crop[categorical].to_dict(orient="records")
X_feb = transform.transform(dict_feb)
y_feb = df_feb_crop["duration"].values
y_pred_feb = model.predict(X_feb)
rmse_feb = mean_squared_error(y_feb, y_pred_feb, squared=False)
print(f"RMSE of the model on February data: {rmse_feb:.2f} minutes")

RMSE of the model on February data: 7.81 minutes


/home/ubuntu/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
